In [ ]:
!pip install -q easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.8 MB/s eta 0:00:00


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from transformers import AutoModelForObjectDetection
from PIL import Image
import easyocr
import re
import pandas as pd
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-detection", revision="no_timm").to(device)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [ ]:
images_folder = "/content/drive/My Drive/diplom/images/sf_png/"
output_csv = "/content/drive/My Drive/diplom/results_2_.csv"

In [ ]:
# Класс для изменения размера изображений
class MaxResize(object):
    def __init__(self, max_size=800):
        self.max_size = max_size

    def __call__(self, image):
        width, height = image.size
        scale = self.max_size / max(width, height)
        return image.resize((int(scale * width), int(scale * height)))

# Трансформации для изображений перед подачей в модель
detection_transform = transforms.Compose([
    MaxResize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Преобразование координат bounding box из центра и размеров в координаты углов
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

# Масштабирование координат bounding box в соответствии с размером изображения
def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

# Добавление метки "no object" в словарь меток модели
id2label = model.config.id2label
id2label[len(model.config.id2label)] = "no object"

# Преобразование выходных данных модели в объекты с метками, вероятностями и координатами
def outputs_to_objects(outputs, img_size, id2label):
    m = outputs.logits.softmax(-1).max(-1)
    pred_labels = list(m.indices.detach().cpu().numpy())[0]
    pred_scores = list(m.values.detach().cpu().numpy())[0]
    pred_bboxes = outputs['pred_boxes'].detach().cpu()[0]
    pred_bboxes = [elem.tolist() for elem in rescale_bboxes(pred_bboxes, img_size)]

    objects = []
    for label, score, bbox in zip(pred_labels, pred_scores, pred_bboxes):
        class_label = id2label[int(label)]
        if class_label != 'no object':
            objects.append({'label': class_label, 'score': float(score), 'bbox': [float(elem) for elem in bbox]})
    return objects

# Функция для обрезки изображений на основе объектов, найденных моделью
def objects_to_crops(img, tokens, objects, class_thresholds, padding=10):
    table_crops = []
    for obj in objects:
        if obj['score'] < class_thresholds.get(obj['label'], 0):
            continue

        bbox = [obj['bbox'][0] - padding, obj['bbox'][1] - padding, obj['bbox'][2] + padding, obj['bbox'][3] + padding]
        cropped_img = img.crop(bbox)

        table_tokens = [token for token in tokens if iob(token['bbox'], bbox) >= 0.5]
        for token in table_tokens:
            token['bbox'] = [token['bbox'][0] - bbox[0], token['bbox'][1] - bbox[1], token['bbox'][2] - bbox[0], token['bbox'][3] - bbox[1]]

        if obj['label'] == 'table rotated':
            cropped_img = cropped_img.rotate(270, expand=True)
            for token in table_tokens:
                bbox = token['bbox']
                bbox = [cropped_img.size[0] - bbox[3] - 1, bbox[0], cropped_img.size[0] - bbox[1] - 1, bbox[2]]
                token['bbox'] = bbox

        table_crops.append({'image': cropped_img, 'tokens': table_tokens})
    return table_crops

In [ ]:
date_patterns = [
    re.compile(r'\b\d{1,2}[.\-/]\d{1,2}[.\-/]\d{2,4}\b'),  # dd.mm.yyyy, dd/mm/yyyy
    re.compile(r'\b\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2}\b')  # yyyy-mm-dd
]

In [ ]:
# Список месяцев на русском языке
months = ["январь", "февраль", "март", "апрель", "май", "июнь", "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"]

# Функция для проверки наличия месяца в тексте
def contains_month(text):
    return any(month in text.lower() for month in months)

# Регулярное выражение для поиска чисел
number_pattern = re.compile(r'\d+[\.,]?\d*')

# Функция для извлечения чисел из текста
def extract_numbers(data):
    numbers = []
    for _, text in data:
        # Игнорируем строки, содержащие месяцы
        if not contains_month(text) and 'г' not in text.lower():
            # Проверяем, не содержит ли строка дату
            if not any(pattern.search(text) for pattern in date_patterns):
                matches = number_pattern.findall(text)
                for match in matches:
                    number = float(match.replace(',', '.'))
                    if 1 <= number <= 20000:  # Допустимый диапазон
                        numbers.append(number)
    return numbers


# Функция для поиска наибольшего числа из списка
def find_energy_amount(numbers):
    return max((value for value in numbers if value <= 9999), default=-1)

# Функция для вычисления центра bounding box
def calculate_center(coordinates):
    x_coords = [point[0] for point in coordinates]
    y_coords = [point[1] for point in coordinates]
    return sum(x_coords) / len(x_coords), sum(y_coords) / len(y_coords)

# Функция для вычисления евклидова расстояния между двумя точками
def euclidean_distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

# Функция для поиска ближайшего значения к ключевому слову по координатам
def find_nearest_value(data, keyword, ignore_value='101528843'):
    keyword_center = None
    values = []

    for item in data:
        coordinates, text, _ = item
        text = correct_text(text)
        center = calculate_center(coordinates)
        if keyword in text:
            keyword_center = center
            matches = re.findall(r'\d{9}', text)
            for match in matches:
                if match != ignore_value:
                    return match
        elif re.match(r'^\d{9}$', text) and text != ignore_value:
            values.append((center, text))

    if not keyword_center or not values:
        return None

    return min(values, key=lambda x: euclidean_distance(keyword_center, x[0]))[1]

# Функция для корректировки текста
def correct_text(text):
    return re.sub(r'УНН', 'УНП', text)

# Функция для сортировки результатов по порогу
def filter_results_by_probability(results, threshold=0.5):
    filtered_results = [result for result in results if result[2] > threshold]
    return filtered_results

reader = easyocr.Reader(['ru'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

In [ ]:
all_results = []
detection_class_thresholds = {
    "table": 0.5,
    "table rotated": 0.5,
    "no object": 10
}

for filename in os.listdir(images_folder):
    file_path = os.path.join(images_folder, filename)
    if not filename.lower().endswith(('.jpg', '.png', '.pdf')):
        continue

    if filename.lower().endswith('.pdf'):
        images_from_pdf = convert_from_path(file_path, first_page=0, last_page=1)
        if images_from_pdf:
            image = images_from_pdf[0]
        else:
            continue
    else:
        image = Image.open(file_path)

    bounds = reader.readtext(file_path)
    UNP = find_nearest_value(bounds, 'УНП')

    pixel_values = detection_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(pixel_values)

    objects = outputs_to_objects(outputs, image.size, id2label)
    tables_crops = objects_to_crops(image, [], objects, detection_class_thresholds)

    if tables_crops:
        cropped_table = tables_crops[0]['image'].convert("RGB")
        table_image_path = f'/content/drive/My Drive/diplom/images/table/table_{filename}.png'
        cropped_table.save(table_image_path)

        bounds_table = reader.readtext(table_image_path)
        filtered_bounds_table = filter_results_by_probability(bounds_table)
        df_table = [(bound[0], bound[1].replace(' ', '')) for bound in filtered_bounds_table]
        # df_table = [(bound[0], bound[1].replace(' ', '')) for bound in bounds_table]

        numbers = extract_numbers(df_table)
        max_number = find_energy_amount(numbers)

        all_results.append({"UNP": UNP, "max_number": max_number})
    else:
        all_results.append({"UNP": UNP, "max_number": None})

df = pd.DataFrame(all_results)
df.to_csv(output_csv, index=False)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [ ]:
df

,UNP,max_number
0,191302295,3502.00
1,None,2075.00
2,600229026,524.00
3,193699894,343.00
4,100742827,81.00
5,100140941,812.59
6,700155928,2728.00
7,100819821,424.00
8,100819821,735.00
9,None,628.73


44/50 унп верно

*   5/6 нет данных об унп
*   1/6 на УНП находится печать, нет возможности прочитать

15/50 дали неправильные значения электроэнергии

*   7/15 таблица опр верно, но функция определяет неправильное число
*   8/15 не определяет таблицу или выдяет неправильно






